In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

### Configure the model training policy


In [2]:
model_development.configure_training_policy()

### Load and preprocess data:

In [3]:
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv')
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

### Prepare data for model training

In [4]:
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


### Train classification model (first phase)


In [5]:
optimizer = model_development.create_optimizer('nadam')
inception_resnetv2 = \
    model_development.inception_resnetv2(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = model_development.train_classification_model(\
    training_phase = 1, model = inception_resnetv2, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'InceptionResNetv2', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 1.0297 - accuracy: 0.6417 - precision: 0.6481 - recall: 0.4705
Epoch 1: val_accuracy improved from -inf to 0.51940, saving model to ../models/weights\InceptionResNetv2_1.00.00_fs.hdf5
119/119 [==============================] - 52s 280ms/step - loss: 1.0297 - accuracy: 0.6417 - precision: 0.6481 - recall: 0.4705 - val_loss: 1.5248 - val_accuracy: 0.5194 - val_precision: 0.4944 - val_recall: 0.5597
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5662 - accuracy: 0.7066 - precision: 0.7678 - recall: 0.5121
Epoch 2: val_accuracy improved from 0.51940 to 0.62090, saving model to ../models/weights\InceptionResNetv2_1.00.00_fs.hdf5
119/119 [==============================] - 27s 229ms/step - loss: 0.5662 - accuracy: 0.7066 - precision: 0.7678 - recall: 0.5121 - val_loss: 0.6598 - val_accuracy: 0.6209 - val_precision: 0.7286 - val_recall: 0.3208
Epoch 3/200
119/119 [==============================] - 

### Train classification model (second phase)


In [6]:
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/InceptionResNetv2_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'InceptionResNetv2', version = '1.00.00_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 270.8374 - accuracy: 0.5657 - precision: 0.5142 - recall: 0.8763
Epoch 1: val_accuracy improved from -inf to 0.57313, saving model to ../models/weights\InceptionResNetv2_1.00.00_sss.hdf5
119/119 [==============================] - 79s 468ms/step - loss: 270.8374 - accuracy: 0.5657 - precision: 0.5142 - recall: 0.8763 - val_loss: 95.0093 - val_accuracy: 0.5731 - val_precision: 0.5323 - val_recall: 0.8302
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 54.5033 - accuracy: 0.6813 - precision: 0.6468 - recall: 0.6647
Epoch 2: val_accuracy did not improve from 0.57313
119/119 [==============================] - 49s 407ms/step - loss: 54.5033 - accuracy: 0.6813 - precision: 0.6468 - recall: 0.6647 - val_loss: 75.0380 - val_accuracy: 0.5164 - val_precision: 0.4932 - val_recall: 0.6792
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 36.0574 - accuracy: 0.6807 - precision: 0.6526 -